# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-04-09 22:42:13,466 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460266932.log


This non-commercial license of GraphLab Create is assigned to yossisynett@gmail.com and will expire on January 09, 2017. For commercial licensing options, visit https://dato.com/buy/.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.383943     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.410458     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
model_all_coef=model_all.get("coefficients")
model_all_coef[model_all_coef['value']!=0]

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [30]:
import numpy as np

def calc_rss(data, output_name, model):
    return (data[output_name] - model.predict(data)).apply(lambda x: x**2).sum()

def linear_regression_with_validation(l1_penalty, train_data, valid_data, output_name, features_list):
    model = graphlab.linear_regression.create(train_data, target=output_name, features=features_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    print(np.count_nonzero(model.get("coefficients")['value']))
    return calc_rss(valid_data, output_name, model)

l1_penalties = np.logspace(1, 7, num=13)
l1_penalty_valid_errors = []
for l1_penalty in l1_penalties:
    l1_penalty_valid_errors.append(
        linear_regression_with_validation(l1_penalty, training, validation, 'price', all_features))

18
18
18
18
18
18
18
18
18
18
18
18
18


In [31]:
l1_penalties_sframe = graphlab.SFrame()
l1_penalties_sframe['penalty'] = l1_penalties
l1_penalties_sframe['valid_errors'] = l1_penalty_valid_errors
l1_penalties_sframe.print_rows(num_rows=13)
l1_penalty_best = l1_penalties[l1_penalty_valid_errors.index(min(l1_penalty_valid_errors))]

+---------------+-------------------+
|    penalty    |    valid_errors   |
+---------------+-------------------+
|      10.0     | 6.25766285142e+14 |
| 31.6227766017 | 6.25766285362e+14 |
|     100.0     | 6.25766286058e+14 |
| 316.227766017 | 6.25766288257e+14 |
|     1000.0    | 6.25766295212e+14 |
| 3162.27766017 | 6.25766317206e+14 |
|    10000.0    | 6.25766386761e+14 |
| 31622.7766017 | 6.25766606749e+14 |
|    100000.0   | 6.25767302792e+14 |
| 316227.766017 | 6.25769507644e+14 |
|   1000000.0   | 6.25776517727e+14 |
| 3162277.66017 | 6.25799062845e+14 |
|   10000000.0  | 6.25883719085e+14 |
+---------------+-------------------+
[13 rows x 2 columns]



*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [32]:
print(l1_penalty_best)
model_best = graphlab.linear_regression.create(training_and_validation, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty_best, verbose = False)
print(calc_rss(testing, 'price', model_best))

10.0
1.59445284798e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [33]:
model_best_coef = model_best.get("coefficients")
print(np.count_nonzero(model_best_coef['value']))

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [34]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [35]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [38]:
l1_penalty_min = min(l1_penalty_values)
l1_penalty_max = max(l1_penalty_values)
last_nonzero = len(all_features)+1
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    nonzero = model['coefficients']['value'].nnz()
    if nonzero > max_nonzeros:
        l1_penalty_min = l1_penalty
    elif nonzero < max_nonzeros:
        l1_penalty_max = l1_penalty
        break

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [44]:
print(l1_penalty_min)
print(l1_penalty_max)

2976351441.63
3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [45]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [65]:
search_results = graphlab.SFrame({'l1_penalty': [], 'valid_rss': [], 'nonzero': graphlab.SArray(dtype=str)})
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    coef = model['coefficients']
    nonzero = coef['value'].nnz()
    if nonzero == max_nonzeros:
        valid_rss = calc_rss(validation, 'price', model)
        nonzero = ','.join(coef[coef['value']>0]['name'])
        print(nonzero)
        search_results = search_results.append(graphlab.SFrame({'l1_penalty': [l1_penalty], 
                                                                'valid_rss': [valid_rss],
                                                                'nonzero': [nonzero]}))

(intercept),bedrooms,bathrooms,sqft_living,sqft_living_sqrt,grade,sqft_above
(intercept),bedrooms,bathrooms,sqft_living,sqft_living_sqrt,grade,sqft_above
(intercept),bedrooms,bathrooms,sqft_living,sqft_living_sqrt,grade,sqft_above
(intercept),bedrooms,bathrooms,sqft_living,sqft_living_sqrt,grade,sqft_above


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [70]:
best_result = search_results[search_results['valid_rss']==search_results['valid_rss'].min()]

In [73]:
print(best_result['l1_penalty'])
print(best_result['valid_rss'])
print(best_result['nonzero'])

[3448968612.163437, ... ]
[1046937488751711.1, ... ]
['(intercept),bedrooms,bathrooms,sqft_living,sqft_living_sqrt,grade,sqft_above', ... ]
